## Introduction
Fraud risk is everywhere, but for companies that advertise online, click fraud can happen at an overwhelming volume, resulting in misleading click data and wasted money. Ad channels can drive up costs by simply clicking on the ad at a large scale. With over 1 billion smart mobile devices in active use every month, China is the largest mobile market in the world and therefore suffers from huge volumes of fradulent traffic.

TalkingData, China’s largest independent big data service platform, covers over 70% of active mobile devices nationwide. They handle 3 billion clicks per day, of which 90% are potentially fraudulent. Their current approach to prevent click fraud for app developers is to measure the journey of a user’s click across their portfolio, and flag IP addresses who produce lots of clicks, but never end up installing apps. With this information, they've built an IP blacklist and device blacklist.

While successful, they want to always be one step ahead of fraudsters and have turned to the Kaggle community for help in further developing their solution. In their 2nd competition with Kaggle, you’re challenged to build an algorithm that predicts whether a user will download an app after clicking a mobile app ad. To support your modeling, they have provided a generous dataset covering approximately 200 million clicks over 4 days!

### Importing the data and python packages

In [1]:
import pandas as pd
import numpy as np
import sklearn
import gc

In [2]:
test_data = pd.read_csv('data/test.csv')
test_data.sample(5)

,click_id,ip,app,device,os,channel,click_time
3804646,3804646,71323,3,1,19,130,2017-11-10 05:08:57
16138412,16138412,97127,9,2,13,442,2017-11-10 14:10:54
8464873,8464873,1456,18,1,17,107,2017-11-10 09:45:49
17689005,17689005,100571,23,1,22,153,2017-11-10 14:38:28
11246353,11246353,39158,9,1,16,334,2017-11-10 10:39:34


In [3]:
train_data = pd.read_csv('data/train.csv', nrows=40000000)
train_data.sample(5)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
1778914,44422,2,1,13,435,2017-11-06 16:43:09,NaN,0
5219709,38265,3,1,17,137,2017-11-06 21:00:22,NaN,0
36837117,90965,3,1,15,280,2017-11-07 08:33:55,NaN,0
36916202,99047,12,1,19,328,2017-11-07 08:35:39,NaN,0
8175864,191642,2,1,13,122,2017-11-06 23:34:45,NaN,0


### Check for missing values

In [4]:
missing_values_count = train_data.isnull().sum()
missing_values_count

ip                        0
app                       0
device                    0
os                        0
channel                   0
click_time                0
attributed_time    39896766
is_attributed             0
dtype: int64

We only have missing values in the atttributed_time. You can choose to drop the columm or fill the missing the values. I will fill the missing data.

In [5]:
train_data = train_data.fillna(0)

In [6]:
# missing test_data
missing_values_count = test_data.isnull().sum()
missing_values_count

click_id      0
ip            0
app           0
device        0
os            0
channel       0
click_time    0
dtype: int64

### Change the datetime field

In [7]:
def change_date(df, time):
    r_time = pd.to_datetime(time)
    df['hour'] = pd.to_datetime(r_time).dt.hour.astype('uint8')
    df['day'] = pd.to_datetime(r_time).dt.day.astype('uint8')
    return df

In [8]:
# datetime on test.data
change_date(test_data, test_data.click_time)
test_data.sample(5)


,click_id,ip,app,device,os,channel,click_time,hour,day
12166024,12166024,46286,18,1,23,376,2017-11-10 10:57:06,10,10
18231471,18231471,47618,12,1,1,242,2017-11-10 14:49:01,14,10
17350573,17350573,5348,3,1,19,442,2017-11-10 14:32:32,14,10
9521792,9521792,112487,17,1,41,128,2017-11-10 10:06:26,10,10
1695495,1695495,29347,12,1,19,178,2017-11-10 04:29:48,4,10


In [9]:
# change on train data
change_date(train_data, train_data.click_time)
train_data.sample(5)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,hour,day
323899,105637,3,1,10,137,2017-11-06 16:06:12,0,0,16,6
11691830,14906,12,1,18,497,2017-11-07 00:39:30,0,0,0,7
3487924,115663,11,1,13,219,2017-11-06 17:54:53,0,0,17,6
39249879,84594,12,1,15,178,2017-11-07 09:24:18,0,0,9,7
29348276,125286,23,1,13,153,2017-11-07 05:58:48,0,0,5,7


### Analyse the feature and group

In [10]:
def group(dataset, g, x):
    gp = dataset.groupby(by=g)[['channel']].count().reset_index().rename(index=str, columns={'channel': x})
    dataset = dataset.merge(gp, on=g, how='left')
    return dataset

In [11]:
# ip-click_time_combination
x = 'ip_tcount'
g = ['ip','day','hour']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [12]:
# ip-app_os combination
x = 'ip_app_os_count'
g = ['ip','app','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [13]:
# ip-device_os combination
x = 'ip_device_os_count'
g = ['ip','device','os']
test_data = group(test_data, g, x )
train_data = group(train_data, g, x)

In [14]:
# ip_app, device combination
x = 'ip_app_device_os_count'
g = ['ip', 'app', 'device', 'os']
test_data = group(test_data, g, x)
train_data = group(train_data, g, x)

In [15]:
#  splitting the data
from sklearn.cross_validation import train_test_split

feature_colums = ['ip','app','device','os', 'channel', 'hour', 'day', 
              'ip_tcount', 'ip_app_os_count',
              'ip_device_os_count', 'ip_app_device_os_count']
label_colums = ['is_attributed']

# Create features and labels
X = train_data[feature_colums].values
y = train_data[label_colums].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train.shape

/Users/rose/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(28000000, 11)

In [ ]:
# training the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, auc, log_loss


model = LogisticRegression()
print('fittinh the model')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
print(y_pred_proba)
# false positive rate, true positive rate, threshold

[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)

print('results')
print("accuracy ", accuracy_score(y_test, y_pred))
print(" log_loss " ,  log_loss(y_test, y_pred_proba))
print(" auc is %2.3f", auc(fpr, tpr))



fittinh the model


/Users/rose/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# create ROC_AUC curve

In [ ]:
# testing the model
test_pred = pd.DataFrame()
test_pred['click_id'] = test_data['click_id'].astype('int')
test_pred.head()

test_pred['is_attributed'] = model.predict(test_data[feature_colums])
test_pred.to_csv('submission_sk.csv', index=False)

In [ ]:
test_pred.head(5)